In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-w9p2/mnist3D_data.json
/kaggle/input/2022-ml-w9p2/submit.csv


In [2]:
%matplotlib inline
import warnings  
warnings.filterwarnings('ignore')

import os
import json
import glob
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objs as go

from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler

In [3]:
with open("../input/2022-ml-w9p2/mnist3D_data.json", 'r') as j:
    data = json.load(j)

In [4]:
# 1720 번째 이미지에 대한 출력
image_number = 1720

digit = data["train"][image_number]["label"]
coordinate = np.array(data["train"][image_number]["points"])

layout = go.Layout(title = "Digit " + str(digit))
plot_data = go.Scatter3d(x = coordinate[:,0], y = coordinate[:,1], z = coordinate[:,2],
                    mode = 'markers', marker = dict(size = 3))

fig = go.Figure(data = [plot_data], layout = layout)
fig.show()

In [5]:
from torch.utils.data import Dataset

class MnistDataset(Dataset):
    def __init__(self, data, mode='train'):
        self.data = data[mode]
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        xyz = np.array(self.data[idx]['points'], dtype=np.float32)
        return {'points':xyz, 'label':self.data[idx]['label']}

In [6]:
# Voxel Grid 생성
# 목표 : point_cloud를 사용자가 지정한 voxel_size 크기의 voxel_grid로 분할한다

def VoxelGrid(points, voxel_size=[1, 1, 1]):

    # Voxel Grid를 구하기 위해 각 축 별 최대, 최소값 계산
    min_xyz  = np.min(points, axis=0) - 0.001
    max_xyz  = np.max(points, axis=0) + 0.001

    diff = max(max_xyz - min_xyz ) - ( max_xyz - min_xyz )
    min_xyz  = min_xyz  - diff / 2
    max_xyz  = max_xyz  + diff / 2

    # voxel_grid
    voxel_grid = []

    # point cloud를 각 축 (x,y,z) 에 대한 voxel grid 로 분할
    for i in range(3):
        # Voxel_size는 정수만 가능
        if type(voxel_size[i]) is not int:
            raise TypeError("voxel_size[{}] must be int".format(i))
        segment = np.linspace( min_xyz[i], max_xyz[i], num=(voxel_size[i] + 1))
        voxel_grid.append(segment)

    return voxel_grid # 생성한 voxel_grid 를 반환

In [7]:
# Voxelization (복셀화) 및 Voxel_Vector 추출
# 목표: 생성한 voxel_grid에 맞추어 point cloud를 각 grid에 맞춰 Voxelization하고, 이를 통해 voxel_vector 

def VoxelVector(points, voxel_grid, voxel_size=[1, 1, 1]):

    n_voxels = voxel_size[0] * voxel_size[1] * voxel_size[2]
    n_x = voxel_size[0]
    n_y = voxel_size[1]
    n_z = voxel_size[2]

    voxelization = np.zeros((len(points), 4), dtype=int)

    # Voxelization
    voxelization[:,0] = np.searchsorted(voxel_grid[0], points[:,0]) - 1
    voxelization[:,1] = np.searchsorted(voxel_grid[1], points[:,1]) - 1
    voxelization[:,2] = np.searchsorted(voxel_grid[2], points[:,2]) - 1

    voxelization[:,3] = ((voxelization[:,1] * n_x) + voxelization[:,0]) + (voxelization[:,2] * (n_x * n_y)) 

    # Voxelization으로부터 voxel_vector 추출
    vector = np.zeros(n_voxels)
    count = np.bincount(voxelization[:,3])
    vector[:len(count)] = count

    vector = vector.reshape(n_z, n_y, n_x)

    return vector # 추출한 Vector 반환


In [8]:
# Voxel_Feature 추출
# 목표: 추출한 voxel_vector로부터 classification에 feature로 활용할 수 있는 voxel_feature를 추출한다

def VoxelFeature(dataset):
    
    voxel_vectors = []
    labels = []

    for i, pc in enumerate(tqdm(dataset)):

        # point cloud를 np.array(dtype=np.float32)로 변경
        pointcloud = np.array(pc['points'], dtype = np.float32)   

        voxel_grid = VoxelGrid(pointcloud, voxel_size=[8, 8, 8])
        
        vector = VoxelVector(pointcloud, voxel_grid, voxel_size=[8, 8, 8])

        voxel_vectors.append(vector.reshape(-1) / np.max(vector))
        
        labels.append(pc['label'])

    return np.array(voxel_vectors), np.array(labels)

In [9]:
# [1] 데이터 불러온 후
mnist_train = MnistDataset(data, mode = "train")
mnist_test = MnistDataset(data, mode = "test")

# [2] 불러온 Raw 데이터를 가공합니다.
x_train, y_train = VoxelFeature(mnist_train)
x_test, _ = VoxelFeature(mnist_test)

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

In [10]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(5000, 512)
(5000,)
(1000, 512)


In [11]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [12]:
from sklearn.preprocessing import MinMaxScaler

mmc = MinMaxScaler()
x_train_mmc = mmc.fit_transform(x_train)
x_test_mmc = mmc.transform(x_test)

In [13]:

submit = pd.read_csv('/kaggle/input/2022-ml-w9p2/submit.csv')

In [14]:
from sklearn.svm import SVC
svc = SVC(C = 3)
svc.fit(x_train_mmc,y_train)
y_pred = svc.predict(x_test_mmc)

In [15]:

submit = pd.read_csv('/kaggle/input/2022-ml-w9p2/submit.csv')
submit

,id,Label
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
995,995,0
996,996,0
997,997,0
998,998,0


In [16]:
submit['Label']= y_pred
submit = submit.to_csv('submit.csv',index =False)